In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE # <-- Importando o SMOTE
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix
)
import matplotlib.pyplot as plt

# ==============================================================================
# --- 1. Carregando e Pré-processando os Dados ---
# ==============================================================================
print("--- 1. Carregando e Pré-processando os Dados ---")

# ... (Seu código de carregamento e pré-processamento) ...
# (Ocultado por brevidade, mas é o mesmo código que você já tem)
# ... (Seu código de carregamento e pré-processamento) ...

# Carregar o dataset
file_path = '../data/raw/leish_dataset.csv'
df = pd.read_csv(file_path)

# Criar cópia para processamento
df_processed = df.copy()

# Lidar com valores ausentes (Missing)
for col in df_processed.select_dtypes(include=['object']).columns:
    df_processed[col] = df_processed[col].fillna('Unknown')

# Codificar a variável Alvo (Target)
target_map = {'positivo': 1, 'negativo': 0, 'Unknown': 0}
df_processed['diagnosis'] = df_processed['diagnosis'].map(target_map).astype(int)

# Separar features (X) e alvo (y)
X_categorical = df_processed.drop('diagnosis', axis=1)
y = df_processed['diagnosis']

# Aplicar One-Hot Encoding nas features categóricas
X_numeric = pd.get_dummies(X_categorical, drop_first=True, dtype=int)
feature_names = X_numeric.columns.tolist() 

print(f"--- Pré-processamento Concluído. {len(feature_names)} features criadas. ---")


# ==============================================================================
# --- 2. Dividindo os Dados (Treino e Teste) ---
# ==============================================================================
print("\n--- 2. Dividindo os Dados (Treino e Teste) ---")

X_train, X_test, y_train, y_test = train_test_split(
    X_numeric, 
    y, 
    test_size=0.3,
    random_state=42,
    stratify=y
)

print(f"Tamanho do conjunto de Treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de Teste:  {X_test.shape[0]}")


# ==============================================================================
# --- 3. Aplicando StandardScaler (Fundamental) ---
# ==============================================================================
print("\n--- 3. Aplicando StandardScaler (ANOS do SMOTE) ---")

# É importante escalar os dados ANTES de aplicar o SMOTE,
# para que o SMOTE calcule as "distâncias" em um espaço padronizado.
scaler = StandardScaler()

# Ajustar o scaler APENAS nos dados de TREINO
X_train_scaled = scaler.fit_transform(X_train)

# Transformar os dados de TESTE com o mesmo scaler
X_test_scaled = scaler.transform(X_test)

print("--- Dados escalados com sucesso ---")


# ==============================================================================
# --- 4. Modelo 9: Aplicando SMOTE ---
# ==============================================================================
print("\n--- 4. Modelo 9: Aplicando SMOTE para balancear os dados de TREINO ---")

# Contar as classes ANTES do SMOTE
unique_before, counts_before = np.unique(y_train, return_counts=True)
print(f"Distribuição de classes ANTES do SMOTE: {dict(zip(unique_before, counts_before))}")

# Inicializar o SMOTE
sm = SMOTE(random_state=42)

# Aplicar SMOTE. IMPORTANTE: Aplicar APENAS nos dados de TREINO (já escalados)
X_train_smote, y_train_smote = sm.fit_resample(X_train_scaled, y_train)

# Contar as classes DEPOIS do SMOTE
unique_after, counts_after = np.unique(y_train_smote, return_counts=True)
print(f"Distribuição de classes DEPOIS do SMOTE: {dict(zip(unique_after, counts_after))}")


# ==============================================================================
# --- 5. Treinando a Regressão Logística com dados do SMOTE ---
# ==============================================================================
print("\n--- 5. Treinando a Regressão Logística com dados do SMOTE ---")

# Inicializar o modelo
# ATENÇÃO: NÃO usamos mais class_weight='balanced', 
# pois os dados de treino JÁ ESTÃO balanceados.
lr_smote_classifier = LogisticRegression(
    random_state=42,
    solver='liblinear' 
)

# Treinar o modelo nos dados de treino (escalados E balanceados)
lr_smote_classifier.fit(X_train_smote, y_train_smote)

print("--- Treinamento do Modelo 9 Concluído ---")


# ==============================================================================
# --- 6. Avaliando o Modelo 9 (LR + SMOTE) ---
# ==============================================================================
print("\n--- 6. Avaliando o Modelo 9 (LR + SMOTE) ---")

# Fazer previsões no conjunto de TESTE (escalado, mas original)
y_pred_lr_smote = lr_smote_classifier.predict(X_test_scaled)

# Calcular a Acurácia
print(f"Acurácia (Modelo 9 - LR + SMOTE): {accuracy_score(y_test, y_pred_lr_smote):.4f}")

# Exibir Relatório de Classificação
print("\nRelatório de Classificação (Modelo 9 - LR + SMOTE):")
print(classification_report(y_test, y_pred_lr_smote, target_names=['negativo (0)', 'positivo (1)']))

# Exibir Matriz de Confusão
print("\nMatriz de Confusão (Modelo 9 - LR + SMOTE):")
cm_lr_smote = confusion_matrix(y_test, y_pred_lr_smote)
print(f"            [Prev. Neg] [Prev. Pos]")
print(f"[Real Neg]  {cm_lr_smote[0][0]:>10} {cm_lr_smote[0][1]:>10}")
print(f"[Real Pos]  {cm_lr_smote[1][0]:>10} {cm_lr_smote[1][1]:>10}")


# ==============================================================================
# --- 7. BÔNUS: Coeficientes do Modelo (SMOTE) ---
# ==============================================================================
print("\n\n--- 7. BÔNUS: Coeficientes da Regressão Logística (Treinada com SMOTE) ---")

# Extrair os coeficientes (pesos) que o modelo aprendeu
coefficients = lr_smote_classifier.coef_[0]

# Criar um DataFrame para visualizar os coeficientes
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient (Peso)': coefficients
})

# Ordenar pelos valores
coef_df = coef_df.sort_values(by='Coefficient (Peso)', ascending=False)

print("\n--- Top 10 Features que MAIS INDICAM 'POSITIVO' (Modelo 9) ---")
print(coef_df.head(10))

print("\n--- Top 10 Features que MAIS INDICAM 'NEGATIVO' (Modelo 9) ---")
print(coef_df.tail(10).sort_values(by='Coefficient (Peso)', ascending=True))

--- 1. Carregando e Pré-processando os Dados ---
--- Pré-processamento Concluído. 43 features criadas. ---

--- 2. Dividindo os Dados (Treino e Teste) ---
Tamanho do conjunto de Treino: 319
Tamanho do conjunto de Teste:  137

--- 3. Aplicando StandardScaler (ANOS do SMOTE) ---
--- Dados escalados com sucesso ---

--- 4. Modelo 9: Aplicando SMOTE para balancear os dados de TREINO ---
Distribuição de classes ANTES do SMOTE: {np.int64(0): np.int64(224), np.int64(1): np.int64(95)}
Distribuição de classes DEPOIS do SMOTE: {np.int64(0): np.int64(224), np.int64(1): np.int64(224)}

--- 5. Treinando a Regressão Logística com dados do SMOTE ---
--- Treinamento do Modelo 9 Concluído ---

--- 6. Avaliando o Modelo 9 (LR + SMOTE) ---
Acurácia (Modelo 9 - LR + SMOTE): 0.6204

Relatório de Classificação (Modelo 9 - LR + SMOTE):
              precision    recall  f1-score   support

negativo (0)       0.78      0.64      0.70        96
positivo (1)       0.41      0.59      0.48        41

    accurac